## Find Data Availability for Stations

Find which RAWS stations in given state have data availability for variables of interest to fuel moisture model.

In [ ]:
# !pip install MesoPy
import pickle
from MesoPy import Meso
import os.path as osp
import os
import pandas as pd
import numpy as np

outpath = "."

meso_token="4192c18707b848299783d59a9317c6e1" # Get your own token...
m=Meso(meso_token)

In [ ]:
os.chdir('..')
from data_funcs import format_raws

In [ ]:
time_start = "202306010800"  # June 1 2022 08:00 in format yyyymmddHHMM
time_s2    = "202306010900"  # small time to get station ids
time_end   = "202306300900"  # June 30 2022 09:00 in format yyyymmddHHMM
state_str = "CO"

vars='air_temp,relative_humidity,precip_accum,fuel_moisture,wind_speed,solar_radiation'

In [ ]:
meso_obss = m.timeseries(start=time_start,end=time_s2, state=state_str, 
                             showemptystations = '0', vars=vars)

In [ ]:
station_df = pd.DataFrame(columns=['STID', 'air_temp', 'relative_humidity', 'precip_accum', 'fuel_moisture', 'wind_speed', 'solar_radiation'],
                  index=range(0, len(meso_obss["STATION"])))

In [ ]:
for i in range(0, station_df.shape[0]):
    station_df["STID"][i] = meso_obss["STATION"][i]["STID"]
    station_df["air_temp"][i] = int("air_temp" in meso_obss["STATION"][i]["SENSOR_VARIABLES"].keys())
    station_df["relative_humidity"][i] = int("relative_humidity" in meso_obss["STATION"][i]["SENSOR_VARIABLES"].keys())
    station_df["precip_accum"][i] = int("precip_accum" in meso_obss["STATION"][i]["SENSOR_VARIABLES"].keys())
    station_df["fuel_moisture"][i] = int("fuel_moisture" in meso_obss["STATION"][i]["SENSOR_VARIABLES"].keys())
    station_df["wind_speed"][i] = int("wind_speed" in meso_obss["STATION"][i]["SENSOR_VARIABLES"].keys())
    station_df["solar_radiation"][i] = int("solar_radiation" in meso_obss["STATION"][i]["SENSOR_VARIABLES"].keys())

In [ ]:
# Filter to stations with complete observations over time period
station_df = station_df[
    (station_df["fuel_moisture"]==1) & 
    (station_df["relative_humidity"]==1) &
    (station_df["precip_accum"]==1) &
    (station_df["air_temp"]==1) &
    (station_df["wind_speed"]==1) &
    (station_df["solar_radiation"]==1)
]
# Extract station IDs

ids = station_df['STID'].tolist()

In [ ]:
print('Number of RAWS Stations: ',station_df.shape[0])

In [ ]:
# write output
# station_df.to_csv(osp.join(outpath, 'station_df_co.csv'), index=False)

## Get Observations

In [ ]:
# Queuery all stations with complete vars
meso_ts = m.timeseries(time_start, time_end, stid=ids, showemptystations = '0', vars=vars)   # ask the object for data

In [ ]:
# Dictionary to be saved for testing
test_dict = {}

In [ ]:
for i in range(0, len(meso_ts['STATION'])):
    raws1 = format_raws(meso_ts['STATION'][i])
    dict1={
        'id': 'case'+str(i+1),
        'time': raws1['time'],
        'rain': raws1['rain'],
        'fm' : raws1['fm'],
        'rh' : raws1['rh'],
        'temp' : raws1['temp'],
        'Ed' : raws1['Ed'],
        'Ew' : raws1['Ew'],
        'wind' : raws1['wind_speed'],
        'solar' : raws1['solar'],
        'STID' : raws1['STID'],
        'title' : 'RAWS Station '+raws1['STID'],
        'descr' : 'RAWS sensor data, Colorado',
        'hours':len(raws1['time']),
        'h2':int(24*20),
        'other': {'lon': raws1['lon'], 'lat': raws1['lat']}
    }
    test_dict['case'+str(i+1)] = dict1 # save to test dictionary

## Save Dictionary

In [ ]:
# Create file name from environment vars
filename = "testing_dict"+"_"+state_str+"_"+time_start[0:6:1]
print(filename)

In [ ]:
os.chdir('data')
with open(filename+'.pickle', 'wb') as handle:
    pickle.dump(test_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)